### 1. 실험 설계를 위한 데이터 분할

#### 1) 데이터 분할을 위한 폴더 생성

In [2]:
import os
import shutil

original_dataset_dir = './leaf_data'
classes_list = os.listdir(original_dataset_dir)

base_dir = './splitted'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for clss in classes_list :
    os.mkdir(os.path.join(train_dir, clss))
    os.mkdir(os.path.join(validation_dir, clss))
    os.mkdir(os.path.join(test_dir, clss))

#### 2) 데이터 분할과 클래스별 데이터 수 확인

- math.floor : 소수점 버림

In [16]:
import math

for clss in classes_list :
    path = os.path.join(original_dataset_dir, clss)
    # path 위치에 존재하는 모든 이미지 파일의 목록을 변수 fnames에 저장
    fnames = os.listdir(path)
    
    train_size = math.floor(len(fnames) * 0.6)
    validation_size =math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    
    # train 데이터에 해당하는 파일의 이름을 train_fnames에 저장
    train_fnames = fnames[:train_size]
    print('Train size(',clss,') : ', len(train_fnames))
    for fname in train_fnames :
        # 복사할 원본 파일의 경로를 지정
        src = os.path.join(path, fname)
        # 복사한 후 저장할 파일의 경로를 지정
        dst = os.path.join(os.path.join(train_dir, clss), fname)
        # src의 경로에 해당하는 파일을 dst의 경로에 저장
        shutil.copyfile(src, dst)
    
    validation_fnames = fnames[train_size : (validation_size + train_size)]
    print('Validation size(',clss,'):', len(validation_fnames))
    for fname in validation_fnames :
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, clss), fname)
        shutil.copyfile(src, dst)
    
    test_fnames = fnames[(train_size + validation_size) : (train_size + validation_size + test_size)]
    print('Test size(',clss,'):', len(test_fnames))
    for fname in test_fnames :
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, clss), fname)
        shutil.copyfile(src, dst)

Train size( Pepper,_bell___healthy ) :  886
Validation size( Pepper,_bell___healthy ): 295
Test size( Pepper,_bell___healthy ): 295
Train size( Grape___Esca_(Black_Measles) ) :  829
Validation size( Grape___Esca_(Black_Measles) ): 276
Test size( Grape___Esca_(Black_Measles) ): 276
Train size( Pepper,_bell___Bacterial_spot ) :  598
Validation size( Pepper,_bell___Bacterial_spot ): 199
Test size( Pepper,_bell___Bacterial_spot ): 199
Train size( Strawberry___healthy ) :  273
Validation size( Strawberry___healthy ): 91
Test size( Strawberry___healthy ): 91
Train size( Grape___Black_rot ) :  708
Validation size( Grape___Black_rot ): 236
Test size( Grape___Black_rot ): 236
Train size( Corn___Common_rust ) :  715
Validation size( Corn___Common_rust ): 238
Test size( Corn___Common_rust ): 238
Train size( Apple___Apple_scab ) :  378
Validation size( Apple___Apple_scab ): 126
Test size( Apple___Apple_scab ): 126
Train size( Potato___healthy ) :  91
Validation size( Potato___healthy ): 30
Test si

- 디버깅 

In [13]:
cd /home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases/leaf_data/Strawberry___Leaf_scorch

/home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases/leaf_data/Strawberry___Leaf_scorch


In [14]:
rm -rf .ipynb_checkpoints

In [15]:
cd /home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases

/home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases


#### 3) 베이스라인 모델 학습을 위한 준비

- transforms.compose() : 이미지 데이터의 전처리, Augmentation 등의 과정에서 사용되는 메서드
        - Augmentation
            - 이미지 데이터의 Augmentation 종류 : 좌우반전, 밝기조절, 이미지 확대 등
            - Augmentation을 통해 이미지에 노이즈를 주어 더욱 강건한 모델 만들 수 있다.
    - transform.Resize([64, 64]) : 이미지 크기를 64*64로 조정
    - transform.ToTensor() : 이미지를 텐서 형태로 변환하고, 모든 값을 0에서 1 사이로 정규화  

- ImageFolder : 데이터셋을 불러오는 메서드
    - 하나의 클래스가 하나의 폴더에 대응되는 구조의 데이터셋을 불러올 때 ImageFolder 메서드를 활용
    - transform 옵션 : 데이터를 불러온 후, 전처리 또는 Augmentation을 할 방법을 지정한다.(앞서 정의한 옵션으로 지정)

- DataLoader : 불러온 이미지 데이터를 주어진 조건에 따라 미니 배치 단위로 분리하는 역할을 수행

In [1]:
import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
BATCH_SIZE = 256
EPOCH = 30


import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
transform_base = transforms.Compose([transforms.Resize((64, 64)),
                                     transforms.ToTensor()])
train_dataset = ImageFolder(root='./splitted/train',
                            transform = transform_base)
val_dataset = ImageFolder(root='./splitted/val',
                         transform = transform_base)


from torch.utils.data import DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = BATCH_SIZE, shuffle=True,
                                           num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                           batch_size = BATCH_SIZE, shuffle=True,
                                           num_workers=4)

In [2]:
cd /home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases/splitted/train

/home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases/splitted/train


In [3]:
rm -rf .ipynb_checkpoints

In [4]:
cd /home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases/splitted/val

/home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases/splitted/val


In [5]:
rm -rf .ipynb_checkpoints

In [6]:
cd /home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases

/home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases


In [7]:
cd /home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases/splitted/test

/home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases/splitted/test


In [8]:
rm -rf .ipynb_checkpoints

In [9]:
cd /home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases

/home/ubuntu/nozzi/pytorch/프로젝트/leaf_diseases


### 2. 베이스라인 모델 설계

#### 1) 베이스라인 모델 설계

- **nn.Moduel**

    - 딥러닝 모델과 관련된 기본 함수를 포함하는 클래스 상속
    - 상속을 통해 nn.Module 클래스에 있는 여러 메서드를 사용할 수 있다.
    - 클래스 내부의 __init__ 함수
        - 모델에서 사용할 모든 Layer를 정의한다. 
        - super()를 통해 nn.Module 내에 있는 메서드를 상속받아 사용한다.
        
- **Conv2d(입력 채널 수, 출력 채널 수, 커널 크기, stride, padding..)**

- **MaxPooling(커널 크기, Stride)**

- **forward 함수**
    - 모델이 학습 데이터를 입력받아 Forward Propagation을 실행시켜 Output을 계산하는 과정을 정의
    
- **ReLU()**
    - Convolution 연산을 통해 생성된 Feature Map 값에 비선형 활성함수 적용
    
- **Dropout**
    - training = self.training 부분은 학습 모드일 때와 검증 모드일 때 각각 다르게 적용되기 위해 존재
    - 학습 과정에서는 드롭아웃을 적용하지만, 평가 과정에서는 적용하지 않는다.
    
- **Softmax()**
    - 각 클래스에 속할 확률을 output으로 출력
    
- **model_base = Net().to(DEVICE)**
    - 정의한 CNN 모델 Net()의 새로운 객체 생성

In [11]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module) :
    def __init__(self) :
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        # x.view 후의 출력 채널 수와 같다.
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 33)
        
    def forward(self, x) :
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)
    
model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

#### 2) 모델 학습을 위한 함수

In [12]:
def train(model, train_lodaer, optimizer) :
    # 입력받는 모델을 학습 모드로 설정
    model.train()
    
    # train_loader에는 (data, target) 형태가 미니 배치 단위로 묶여 있다.
    # train_loader에 enumerate() 함수를 적용하여 아래와 같은 형태로 반복 가능한 객체가 생성되어 for문 실행
    for batch_idx, (data, target) in enumerate(train_loader) :
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        # loss값을 바탕으로 Back Propagation을 통해 계산한 Gradient값을 각 parameter에 할당
        loss.backward()
        # parameter 업데이트
        optimizer.step()

In [13]:
input = torch.randn(3, 64, 64)
model = Net()
for batch_idx, (data, target) in enumerate(train_loader) :
    output = model(data)
    loss = F.cross_entropy(output, target)

In [14]:
print(output.shape)
b = output.view(-1)
b.shape

torch.Size([182, 33])


torch.Size([6006])

#### 3) 모델 평가를 위한 함수

- (target.view_as(pred)
   - target tensor의 구조를 pred tensor와 같은 모양으로 정렬
   - view() : 정렬하고 싶은 tensor의 모양을 숫자로 직접 지정
   - target.shape : torch.Size([182])
   - pred.shape : torch.Size([182, 1])

In [16]:
def evaluate(model, test_loader) :
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad() :
        for data, target in test_loader : 
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

#### 4) 모델 학습 실행하기

- copy.deepcopy(model.state_dict())
    - 만약 (검증 손실(validation loss) 결과에 따라) 가장 성능이 좋은 모델만 유지할 계획이라면, 
    - best_model_state = model.state_dict() 은 모델의 복사본이 아닌 모델의 현재 상태에 대한 참조(reference)만 반환한다는 사실을 잊으시면 안됨 !
    - 따라서 best_model_state 을 직렬화(serialize)하거나, best_model_state = deepcopy(model.state_dict()) 을 사용해야 한다.
    - 그렇지 않으면, 제일 좋은 성능을 내는 best_model_state 은 계속되는 학습 단계에서 갱신될 것입니다. 결과적으로, 최종 모델의 상태는 과적합(overfit)된 상태가 됩니다.

In [10]:
!nvidia-smi

Thu Dec 29 18:38:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   41C    P0    63W / 300W |   2703MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

In [17]:
import time
import copy

def train_baseline(model, train_loader, val_loader, optimizer, num_epochs=30) :
    best_acc = 0.0
    # 정확도가 가장 높은 모델을 저장할 변수 선언
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs+1) :
        # 한 epoch당 소요되는 시간을 측정하기 위해 epoch 시작할 때의 시각 저장
        since = time.time()
        # 모델 학습
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)
        
        # 현재 epoch의 검증 정확도가 최고 정확도보다 높다면 업데이트 & 해당 epoch의 모델 저장
        if val_acc > best_acc :
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
        # 소요되는 시간 계산
        time_elapsed = time.time() - since
        print('------------------------epoch {}------------------------'.format(epoch))
        print('train Loss : {:.4f}, Accuracy: {:.2f}%'
              .format(train_loss, train_acc))
        print('val Loss : {:.4f}, Accuracy: {:.2f}%'
              .format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'
              .format(time_elapsed // 60, time_elapsed % 60))
        

    # 정확도가 가장 높은 모델을 불러온 뒤, 반환
    model.load_state_dict(best_model_wts)
    return model

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)
torch.save(base, 'baseline.pt')

------------------------epoch 1------------------------
train Loss : 1.5898, Accuracy: 54.74%
val Loss : 1.6060, Accuracy: 54.41%
Completed in 0m 29s
------------------------epoch 2------------------------
train Loss : 0.9711, Accuracy: 70.76%
val Loss : 0.9914, Accuracy: 70.10%
Completed in 0m 29s
------------------------epoch 3------------------------
train Loss : 0.7688, Accuracy: 76.13%
val Loss : 0.8002, Accuracy: 74.56%
Completed in 0m 29s
------------------------epoch 4------------------------
train Loss : 0.5314, Accuracy: 83.99%
val Loss : 0.5684, Accuracy: 82.22%
Completed in 0m 29s
------------------------epoch 5------------------------
train Loss : 0.4778, Accuracy: 85.35%
val Loss : 0.5277, Accuracy: 83.09%
Completed in 0m 29s
------------------------epoch 6------------------------
train Loss : 0.4422, Accuracy: 86.21%
val Loss : 0.4959, Accuracy: 83.78%
Completed in 0m 29s
------------------------epoch 7------------------------
train Loss : 0.3967, Accuracy: 87.46%
val Lo

### 3. Transfer Learning

#### 1) Transfer Learning을 위한 준비(데이터)

In [18]:
import os
import shutil

data_transforms={
    'train' : transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    
    'val' : transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

data_dir = './splitted'
image_datasets = {x : ImageFolder(root=os.path.join(data_dir, x), 
                                  transform=data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x : torch.utils.data.DataLoader(image_datasets[x], batch_size = BATCH_SIZE, shuffle=True, num_workers=4)
               for x in ['train', 'val']}

dataset_sizes = {x : len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

#### 2) Pre-trained Model 불러오기

In [19]:
from torchvision import models

resnet = models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 33)
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(filter(lambda p : p.requires_grad,
                                 resnet.parameters()), lr=0.001)

from torch.optim import lr_scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

#### 3) Pre-trained Model의 일부 Layer Freeze

- `children()` : 모델의 자식 모듈을 반복 가능한 객체로 반환하는 메서드
- `resnet.children()` : 생성한 resnet 모델의 모든 layer 정보를 담고 있다.
- `child.parameters()` : 각 layer의 parameter tensor
    - 각 텐셔에는 `requires_grad` 옵션이 있고 디폴트 값은 True
    - `requires_grad = False` : 파라미터 업데이트 되지 않도록

In [20]:
ct = 0
for child in resnet.children() :
    ct += 1
    if ct < 6 :
        for param in child.parameters() :
            param.requires_grad = False

#### 4) Transfer Learning 모델 학습과 검증을 위한 함수

In [32]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25) :
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs) :
        print('-----------------epoch {}-----------------'.format(epoch+1))
        since = time.time()
        
        for phase in ['train', 'val'] :
            if phase == 'train' :
                model.train()
            else :
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase] :
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train') :
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train' :
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train' :
                scheduler.step()
                l_r = [x['lr'] for x in optimizer_ft.param_groups]
                print('learning rate : ', l_r)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double()/dataset_sizes[phase]
            
            print('{} Loss : {:.4f} Acc : {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc :
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        time_elapsed = time.time() - since
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed%60))
            
    print('Best val Acc : {:4f}'.format(best_acc))
        
    model.load_state_dict(best_model_wts)
        
    return model

#### 5) 모델 학습 실행하기

In [33]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH)
torch.save(model_resnet50, 'resnet50.pt')

-----------------epoch 1-----------------
learning rate :  [0.001]
train Loss : 0.1276 Acc : 0.9591
val Loss : 0.1640 Acc : 0.9498
Completed in 0m 19s
-----------------epoch 2-----------------
learning rate :  [0.001]
train Loss : 0.1148 Acc : 0.9637
val Loss : 0.2473 Acc : 0.9240
Completed in 0m 19s
-----------------epoch 3-----------------
learning rate :  [0.001]
train Loss : 0.1104 Acc : 0.9640
val Loss : 0.1690 Acc : 0.9492
Completed in 0m 19s
-----------------epoch 4-----------------
learning rate :  [0.0001]
train Loss : 0.0862 Acc : 0.9712
val Loss : 0.0940 Acc : 0.9693
Completed in 0m 19s
-----------------epoch 5-----------------
learning rate :  [0.0001]
train Loss : 0.0438 Acc : 0.9856
val Loss : 0.0489 Acc : 0.9840
Completed in 0m 19s
-----------------epoch 6-----------------
learning rate :  [0.0001]
train Loss : 0.0288 Acc : 0.9907
val Loss : 0.0465 Acc : 0.9857
Completed in 0m 19s
-----------------epoch 7-----------------
learning rate :  [0.0001]
train Loss : 0.0238 Acc

### 4. 모델 평가

#### 1) 베이스라인 모델 평가를 위한 전처리

In [26]:
# 어떻게 조정할지 ?
transform_base = transforms.Compose([transforms.Resize([64, 64]),
                                     transforms.ToTensor()])

# 데이터 불러오고 위에서 정의한 transform 적용하고
test_base = ImageFolder(root = './splitted/test',
                        transform = transform_base)

# 데이터 로더로 배치, 셔플 등
test_loader_base = torch.utils.data.DataLoader(test_base,
                                               batch_size = BATCH_SIZE, shuffle=True, num_workers=4)

#### 2) Transfer learning 모델 평가를 위한 전처리

In [28]:
# transform 어떻게 조정할지 ?
transform_resNet = transforms.Compose([transforms.Resize([64, 64]),
                                       transforms.RandomCrop(52),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])
                                      ])

# 데이터 불러오고 위에서 정의한 transform 적용하고
test_resNet = ImageFolder(root='./splitted/test',
                          transform = transform_resNet)
                          
# 데이터 로더로 배치, 셔플 등
test_loader_resNet = torch.utils.data.DataLoader(test_resNet,
                                                 batch_size = BATCH_SIZE, shuffle=True, num_workers=4)

#### 3) 베이스라인 모델 성능 평가하기

In [30]:
baseline = torch.load('baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('baseline test acc : ', test_accuracy)

baseline test acc :  94.14341133775497


#### 4) Transfer Learning 모델 성능 평가하기

In [34]:
resnet50 = torch.load('resnet50.pt')
resnet50.eval()
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('ResNet test acc : ', test_accuracy)

ResNet test acc :  99.04893004630209


---------------------------------------------------------------------------------------------------

### 결과
- 모델을 직접 구축하여 처음부터 학습시키는 것보다 많은 양의 데이터셋으로 미리 학습된 모델을 불러와 일부를 Fine-Tuning 하는 것이 더 높은 예측 성능을 낸다.